## BDCC Assignment: Mini Project -  Indian Premier League (IPL) Data Analysis

#### Team Members

12120067 - Bhargavi Peddapati <br>
12120019 - Sriya Paruchuri <br>
12120045 - Madhab Chakraborty <br>
12120095 - Nararaj G T <br>
12120087 - Rama Gangadhar Durvasula <br>

### Section - Dataset

**Objective :** Select a reasonably sized dataset with at least two files and one file having at least 6 columns and 1 MB in size, define a schema and load the data into the pipeline while ensuring handling of bad records. <br>
**Approach  :** We have searched the dataset in Kaggle site. Defined custom schema and loaded data from dbfs:/FileStore/tables/ipl_datasets and loaded into the pipeline

d
#### IPL Dataset Description <br>



We have selected Indian Premier League (IPL) dataset from Kaggle site. Here, is the link for the site to download the dataset.

- https://github.com/Bhargavi-6/BDCC_Group1_Assignment/blob/a22aa23bda2f908bdadc6aabeda36c4a1dea4b64/datasets/matches.csv
- https://github.com/Bhargavi-6/BDCC_Group1_Assignment/blob/a22aa23bda2f908bdadc6aabeda36c4a1dea4b64/datasets/deliveries.csv <br>

This dataset contains information about the matches and players in the Indian Premier League (IPL) matches between 2008 and 2016. <br>
It has two CSV files, one with data on matches and another with data on players. 
<br>

  - **matches.csv with size 117.1 KB and 18 number of columns**
  - **deliveries.csv with size 15.44 MB and 21 number of columns**
  
##### Description on matches.csv - This dataset contains each match information


- id: A unique identifier for each match played in IPL.
- season: The year in which the IPL tournament was played.
- city: The name of the city where the match was played.
- date: The date on which the match was played.
- team1: The name of the first team playing in the match.
- team2: The name of the second team playing in the match.
- toss_winner: The name of the team that won the toss.
- toss_decision: The decision made by the team that won the toss, whether to bat or field.
- result: The result of the match, whether it was a win, loss or a tie.
- dl_applied: A binary field indicating whether the Duckworth-Lewis (DL) method was applied to the match. The DL method is a mathematical formula used to calculate the target score for the team batting second in a rain-affected match.
- winner: The name of the winning team in the match.
- win_by_runs: The number of runs by which the winning team won the match.
- win_by_wickets: The number of wickets by which the team won.
- player_of_match: The name of the player who played well in the match.
- venue: The name of the stadium where the match was played.
- umpire1: The name of the first umpire who officiated the match.
- umpire2: The name of the second umpire who officiated the match.
- umpire3: The name of the third umpire who officiated the match (if any).


##### Description on deliveries.csv - This dataset contains each ball details for all the seasons matches

- match_id: A unique identifier for each match played in IPL.
- inning: The inning number of the match (1st or 2nd).
- batting_team: The name of the team that is currently batting.
- bowling_team: The name of the team that is currently bowling.
- over: The over number of the match.
- ball: The ball number of the over.
- batsman: The name of the batsman who is currently facing the ball.
- non_striker: The name of the non-striking batsman.
- bowler: The name of the bowler who is currently bowling.
- is_super_over: A binary field indicating whether the current over is a Super Over.
- wide_runs: The number of runs scored on a wide delivery.
- bye_runs: The number of runs scored on a bye.
- legbye_runs: The number of runs scored on a leg bye.
- noball_runs: The number of runs scored on a no-ball delivery.
- penalty_runs: The number of runs scored on a penalty.
- batsman_runs: The number of runs scored by the batsman on this delivery.
- extra_runs: The total number of extra runs scored on this delivery (wide, bye, leg bye, no ball, penalty).
- total_runs: The total number of runs scored on this delivery (batsman runs + extra runs).
- player_dismissed: The name of the batsman who got out on this delivery (if any).
- dismissal_kind: The type of dismissal on this delivery (if any), such as caught, bowled, run out, etc.
- fielder: The name of the fielder who caught or ran out the batsman



#### Loading data onto DBFS

- Go to **Data** on the left menu
- Click on **Create Table**
- Add **ipl_datasets** to /FileStore/tables directory path
- Drag and drop files or Upload the file onto the DBFS location: **/FileStore/tables/tables/ipl_datasets/** under **Files** section.

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

#### Loading the dataset

- Data is provided in *csv* format. Spark provides *sqlContext.read.csv* to read the csv data and create the dataframe. 
- Spark can automatically infer the schema from the data if *inferSchema* is set to True. But we are defining custom schema.

In [0]:
%fs ls /FileStore/tables/ipl_datasets/

path,name,size,modificationTime
dbfs:/FileStore/tables/ipl_datasets/deliveries.csv,deliveries.csv,15442270,1678101160000
dbfs:/FileStore/tables/ipl_datasets/matches.csv,matches.csv,117096,1678101115000


### Define schema for matches.csv

In [0]:
from pyspark.sql.types import *

# Define schema for matches.csv
matches_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("season", IntegerType(), True),
    StructField("city", StringType(), True),
    StructField("date", StringType(), True),
    StructField("team1", StringType(), True),
    StructField("team2", StringType(), True),
    StructField("toss_winner", StringType(), True),
    StructField("toss_decision", StringType(), True),
    StructField("result", StringType(), True),
    StructField("dl_applied", IntegerType(), True),
    StructField("winner", StringType(), True),
    StructField("win_by_runs", IntegerType(), True),
    StructField("win_by_wickets", IntegerType(), True),
    StructField("player_of_match", StringType(), True),
    StructField("venue", StringType(), True),
    StructField("umpire1", StringType(), True),
    StructField("umpire2", StringType(), True),
    StructField("umpire3", StringType(), True)
])


In [0]:
matches_df = sqlContext.read.csv("dbfs:/FileStore/tables/ipl_datasets/matches.csv", 
                                header = True, 
                                schema = StructType(matches_schema) )
matches_df.cache()
matches_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- season: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- date: string (nullable = true)
 |-- team1: string (nullable = true)
 |-- team2: string (nullable = true)
 |-- toss_winner: string (nullable = true)
 |-- toss_decision: string (nullable = true)
 |-- result: string (nullable = true)
 |-- dl_applied: integer (nullable = true)
 |-- winner: string (nullable = true)
 |-- win_by_runs: integer (nullable = true)
 |-- win_by_wickets: integer (nullable = true)
 |-- player_of_match: string (nullable = true)
 |-- venue: string (nullable = true)
 |-- umpire1: string (nullable = true)
 |-- umpire2: string (nullable = true)
 |-- umpire3: string (nullable = true)



In [0]:
matches_df.show(5)

+---+------+---------+----------+--------------------+--------------------+--------------------+-------------+------+----------+--------------------+-----------+--------------+---------------+--------------------+--------------+-------------+-------+
| id|season|     city|      date|               team1|               team2|         toss_winner|toss_decision|result|dl_applied|              winner|win_by_runs|win_by_wickets|player_of_match|               venue|       umpire1|      umpire2|umpire3|
+---+------+---------+----------+--------------------+--------------------+--------------------+-------------+------+----------+--------------------+-----------+--------------+---------------+--------------------+--------------+-------------+-------+
|  1|  2017|Hyderabad|2017-04-05| Sunrisers Hyderabad|Royal Challengers...|Royal Challengers...|        field|normal|         0| Sunrisers Hyderabad|         35|             0|   Yuvraj Singh|Rajiv Gandhi Inte...|   AY Dandekar|     NJ Llong|   nu

#### Memory Consumed to store the matches dataframe

In [0]:
spark._jvm.org.apache.spark.util.SizeEstimator.estimate(matches_df._jdf)

Out[52]: 151316936

#### Partitions the matches dataframe has

The dataframes are internally stores as RDDs and partitions. To find number of partitions.

In [0]:
matches_df.rdd.getNumPartitions()

Out[53]: 1

### Define schema for deliveries.csv

In [0]:
# Define schema for deliveries.csv
deliveries_schema = StructType([
    StructField("match_id", IntegerType(), True),
    StructField("inning", IntegerType(), True),
    StructField("batting_team", StringType(), True),
    StructField("bowling_team", StringType(), True),
    StructField("over", IntegerType(), True),
    StructField("ball", IntegerType(), True),
    StructField("batsman", StringType(), True),
    StructField("non_striker", StringType(), True),
    StructField("bowler", StringType(), True),
    StructField("is_super_over", IntegerType(), True),
    StructField("wide_runs", IntegerType(), True),
    StructField("bye_runs", IntegerType(), True),
    StructField("legbye_runs", IntegerType(), True),
    StructField("noball_runs", IntegerType(), True),
    StructField("penalty_runs", IntegerType(), True),
    StructField("batsman_runs", IntegerType(), True),
    StructField("extra_runs", IntegerType(), True),
    StructField("total_runs", IntegerType(), True),
    StructField("player_dismissed", StringType(), True),
    StructField("dismissal_kind", StringType(), True),
    StructField("fielder", StringType(), True)
])

In [0]:
deliveries_df = sqlContext.read.csv("dbfs:/FileStore/tables/ipl_datasets/deliveries.csv", 
                                header = True, 
                                schema = StructType(deliveries_schema) )
deliveries_df.cache()
deliveries_df.printSchema()

root
 |-- match_id: integer (nullable = true)
 |-- inning: integer (nullable = true)
 |-- batting_team: string (nullable = true)
 |-- bowling_team: string (nullable = true)
 |-- over: integer (nullable = true)
 |-- ball: integer (nullable = true)
 |-- batsman: string (nullable = true)
 |-- non_striker: string (nullable = true)
 |-- bowler: string (nullable = true)
 |-- is_super_over: integer (nullable = true)
 |-- wide_runs: integer (nullable = true)
 |-- bye_runs: integer (nullable = true)
 |-- legbye_runs: integer (nullable = true)
 |-- noball_runs: integer (nullable = true)
 |-- penalty_runs: integer (nullable = true)
 |-- batsman_runs: integer (nullable = true)
 |-- extra_runs: integer (nullable = true)
 |-- total_runs: integer (nullable = true)
 |-- player_dismissed: string (nullable = true)
 |-- dismissal_kind: string (nullable = true)
 |-- fielder: string (nullable = true)



#### Memory Consumed to store the deliveries dataframe

In [0]:
spark._jvm.org.apache.spark.util.SizeEstimator.estimate(deliveries_df._jdf)

Out[56]: 151727056

#### Partitions the deliveries dataframe has

The dataframes are internally stores as RDDs and partitions. To find number of partitions.

In [0]:
deliveries_df.rdd.getNumPartitions()

Out[57]: 4

#### Define the pipeline

In [0]:

dbutils.fs.rm("dbfs:/user/hive/warehouse/matches", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/deliveries", True)


Out[58]: True

In [0]:
from pyspark.sql.functions import col, round, sum

matches_df.write.mode("overwrite").saveAsTable("matches")
deliveries_df.write.mode("overwrite").saveAsTable("deliveries")